<a href="https://www.kaggle.com/code/adithyalbhat/nlp-basics111?scriptVersionId=134086750" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import fastai
from fastai.text.all import *
path=untar_data(URLs.IMDB)

In [3]:
!ls {path}

README	imdb.vocab  test  tmp_clas  tmp_lm  train  unsup


In [4]:
files=get_text_files(path,folders=['train','test','unsup'])

In [5]:
txt=files[0].open().read();
txt[:75]

'Jiang Xian uses the complex backstory of Ling Ling and Mao Daobing to study'

In [6]:
spacy=WordTokenizer()
toks=first(spacy([txt]))
print(coll_repr(toks,30))

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

(#143) ['Jiang','Xian','uses','the','complex','backstory','of','Ling','Ling','and','Mao','Daobing','to','study','Mao',"'s",'"','cultural','revolution','"','(','1966','-','1976',')','at','the','village','level','.'...]


In [7]:
tkn=Tokenizer(spacy)
print(coll_repr(tkn(txt),30))

(#158) ['xxbos','xxmaj','jiang','xxmaj','xian','uses','the','complex','backstory','of','xxmaj','ling','xxmaj','ling','and','xxmaj','mao','xxmaj','daobing','to','study','xxmaj','mao',"'s",'"','cultural','revolution','"','(','1966'...]


In [8]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [9]:
txts= L(o.open().read() for o in files[:2000])

In [10]:
def subword(sz):
    sp=SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp(txts))[:40])

In [11]:
subword(1000)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tmp/texts.out --vocab_size=1000 --model_prefix=tmp/spm --character_coverage=0.99999 --model_type=unigram --unk_id=9 --pad_id=-1 --bos_id=-1 --eos_id=-1 --minloglevel=2 --user_defined_symbols=▁xxunk,▁xxpad,▁xxbos,▁xxeos,▁xxfld,▁xxrep,▁xxwrep,▁xxup,▁xxmaj --hard_vocab_limit=false


'▁J i ang ▁ X ian ▁us es ▁the ▁comp le x ▁back st or y ▁of ▁L ing ▁L ing ▁and ▁Ma o ▁Da o b ing ▁to ▁st u d y ▁Ma o \' s ▁" c ul'

In [12]:
subword(200)

'▁ J i an g ▁ X i an ▁ us es ▁the ▁ com p le x ▁b a ck s t or y ▁of ▁ L ing ▁ L ing ▁and ▁M a o ▁ D a o'

In [13]:
toks
tkn

Tokenizer:
encodes: (Path,object) -> encodes
(str,object) -> encodes
decodes: (object,object) -> decodes

In [14]:
toks=tkn(txt)
print(coll_repr(toks))

(#158) ['xxbos','xxmaj','jiang','xxmaj','xian','uses','the','complex','backstory','of'...]


In [15]:
toks200=txts[:200].map(tkn)

In [16]:
num=Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2152) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

In [17]:
nums=num(toks)
nums[:20]

TensorText([   2,    8,    0,    8,    0, 1269,    9, 1270,    0,   14,    8,
               0,    8,    0,   12,    8,    0,    8,    0,   15])

In [18]:


' '.join(num.vocab[o] for o in nums)
     


'xxbos xxmaj xxunk xxmaj xxunk uses the complex xxunk of xxmaj xxunk xxmaj xxunk and xxmaj xxunk xxmaj xxunk to study xxmaj xxunk \'s " xxunk revolution " ( xxunk - xxunk ) at the xxunk level . xxmaj the film has the elements and pace of xxmaj xxunk opera and so appears slow and sometimes xxunk to the xxunk viewer . xxmaj but the movie provides a window onto xxunk life in xxmaj china , with its focus upon xxunk in the city , the xxunk quality of xxunk -- daily struggle , family and local xxunk -- and the appeal of movies as escape , fantasy , and , ultimately , as xxunk of xxunk . xxmaj this last is the most xxunk element in the film , for it suggests the modern -- and universal -- experience of culture will xxunk the xxunk xxmaj xxunk xxunk . xxmaj the child actors are particularly fine .'

In [19]:
streams = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."

In [20]:
from IPython.core.display import HTML
tokens=tkn(streams)
bs,seq_len=6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [21]:


#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
print(df)
display(HTML(df.to_html(index=False,header=None)))
     
print(np.array(tokens[1]))
np.array(Tokens[i*15:i*15+seq_len] for i in range(6))

       0        1             2        3        4
0  xxbos    xxmaj            in     this  chapter
1  movie  reviews            we  studied       in
2  first       we          will     look       at
3    how       to     customize       it        .
4     of      the  preprocessor     used       in
5   will    study           how       we    build


xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


xxmaj


array(<generator object <genexpr> at 0x793311b0b760>, dtype=object)

In [22]:
nums200=toks200.map(num)

#### 

In [23]:
dl=LMDataLoader(nums200)

In [24]:
x,y=first(dl)

In [25]:
y

TensorText([[   8,    0,    8,  ...,    0,  128,   17],
            [ 240,   12,    0,  ...,  325,    0,   14],
            [  10,   13,    0,  ...,   24,  186,    9],
            ...,
            [ 203,  943,   10,  ...,   34,    9, 1229],
            [  10,   29,    9,  ...,    0,  255,   55],
            [   0,   10,    0,  ...,    8,   33,   60]])

In [26]:
' '.join(num.vocab[o] for o in x[0][:20])
#' '.join(num.vocab[o] for o in x[0][:20])
     


'xxbos xxmaj xxunk xxmaj xxunk uses the complex xxunk of xxmaj xxunk xxmaj xxunk and xxmaj xxunk xxmaj xxunk to'

In [27]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj xxunk xxmaj xxunk uses the complex xxunk of xxmaj xxunk xxmaj xxunk and xxmaj xxunk xxmaj xxunk to study'

In [28]:
get_imdb=partial(get_text_files,folders=['train', 'test', 'unsup'])
dls_lm=DataBlock(blocks=TextBlock.from_folder(path,is_lm=True),
                get_items=get_imdb,
                splitter=RandomSplitter(0.1)
                ).dataloaders(path,path=path,bs=128,seq_len=80)

In [29]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos i have to say although i despise these kind of shows , shock horror , xxmaj i 'm a girl , i feel i have to express my opinion . i had seen xxmaj dirty xxmaj sanchez before i saw xxmaj jackass and think it way surpasses xxmaj jackass in terms of programme making . xxmaj story lines and interviews are inter weaved to create a more interesting show . i saw a few minutes of xxmaj jackass movie","i have to say although i despise these kind of shows , shock horror , xxmaj i 'm a girl , i feel i have to express my opinion . i had seen xxmaj dirty xxmaj sanchez before i saw xxmaj jackass and think it way surpasses xxmaj jackass in terms of programme making . xxmaj story lines and interviews are inter weaved to create a more interesting show . i saw a few minutes of xxmaj jackass movie the"
1,"actions . \n\n xxmaj when i first heard of this show , it did n't caught my attention at all . xxmaj it seemed too ordinary , actually . xxmaj then , i saw some episodes … and loved it ! xxmaj first of all , the characters . xxmaj they are all well - written and different from each other . xxmaj there 's a alcohol addicted , a woman whose fiancée cheats on her , a woman who",". \n\n xxmaj when i first heard of this show , it did n't caught my attention at all . xxmaj it seemed too ordinary , actually . xxmaj then , i saw some episodes … and loved it ! xxmaj first of all , the characters . xxmaj they are all well - written and different from each other . xxmaj there 's a alcohol addicted , a woman whose fiancée cheats on her , a woman who just"


In [30]:
learn=language_model_learner(dls_lm,AWD_LSTM,drop_mult=0.3,
                            metrics=[accuracy,Perplexity()]).to_fp16()

In [31]:
learn.fit_one_cycle(1,2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.005880,3.897072,0.300580,49.258026,29:27


In [32]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [33]:
learn=learn.load('1epoch')

In [34]:
learn.unfreeze()
learn.fit_one_cycle(10,2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.768658,3.757037,0.316509,42.821377,30:24
1,3.701975,3.710211,0.322070,40.862411,30:37
2,3.612359,3.650167,0.328549,38.481083,30:49
3,3.558496,3.617464,0.332600,37.243004,30:54
4,3.488441,3.595127,0.335284,36.420330,31:14
5,3.430982,3.578177,0.337696,35.808220,31:04
6,3.362235,3.569434,0.339194,35.496510,31:51
7,3.290068,3.567074,0.340044,35.412807,31:50
8,3.251925,3.569489,0.340513,35.498459,32:22
9,3.212075,3.574036,0.340244,35.660225,32:10


In [35]:
learn.save_encoder('finetuned')

In [36]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [37]:
print("\n".join(preds))

i liked this movie because it was so nice to see the characters Ryan Stiles and Jennifer Aniston play their lead characters . The acting is great , the humor is sharp and sensitive and the plot is very clever
i liked this movie because the Parker and Parker films are great . In fact , i think the most recent Parker Posey movie is " a Clockwork Orange " , which is about as sexy as ever


In [38]:
dls_clas=DataBlock(
                    blocks=(TextBlock.from_folder(path,vocab=dls_lm.vocab),CategoryBlock),
                    get_y=parent_label,
                    get_items=partial(get_text_files,folders=['train','test']),
                    splitter=GrandparentSplitter(valid_name='test')).dataloaders(path,path=path,bs=128,seq_len=72)

In [39]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxrep 3 * xxmaj warning - this review contains "" plot spoilers , "" though nothing could "" spoil "" this movie any more than it already is . xxmaj it really xxup is that bad . xxrep 3 * \n\n xxmaj before i begin , xxmaj i 'd like to let everyone know that this definitely is one of those so - incredibly - bad - that - you - fall - over - laughing movies . xxmaj if you 're in a lighthearted mood and need a very hearty laugh , this is the movie for you . xxmaj now without further ado , my review : \n\n xxmaj this movie was found in a bargain bin at wal - mart . xxmaj that should be the first clue as to how good of a movie it is . xxmaj secondly , it stars the lame action",neg
2,"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n\n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of oatmeal . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj victor xxmaj vargas became i was always aware that something did n't quite feel right . xxmaj victor xxmaj vargas suffers from a certain overconfidence on the director 's part . xxmaj apparently , the director thought that the ethnic backdrop of a xxmaj latino family on the lower east side , and an idyllic storyline would make the film critic proof . xxmaj he was right , but it did n't fool me . xxmaj raising xxmaj victor xxmaj vargas is",neg
